<a href="https://colab.research.google.com/github/Tekri96/spotify_etl_pipeline_aws/blob/main/spotifydataanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spotipy
!pip install pandas

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
#authentication
client_credentials_manager = SpotifyClientCredentials(client_id="Enter your clientid", client_secret="Enter your client secret")

In [ ]:
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [ ]:
#top songs Canada weekly
playlist_link = "https://open.spotify.com/playlist/37i9dQZEVXbMda2apknTqH"

In [ ]:
playlist_URI = playlist_link.split("/")[-1]
playlist_URI

'37i9dQZEVXbMda2apknTqH'

In [ ]:
data = sp.playlist_tracks(playlist_URI)
#data


In [ ]:
data['items'][0]['track']['album']['name']

'One Thing At A Time'

In [ ]:
album_list = []
for row in data['items']:
    album_id = row['track']['album']['id']
    album_name = row['track']['album']['name']
    album_release_date = row['track']['album']['release_date']
    album_total_tracks = row['track']['album']['total_tracks']
    album_url = row['track']['album']['external_urls']['spotify']
    album_element = {'album_id':album_id,'name':album_name,'release_date':album_release_date,
                        'total_tracks':album_total_tracks,'url':album_url}
    album_list.append(album_element)

In [ ]:
#album_list

In [ ]:
data['items'][0]['track']['artists']


[{'external_urls': {'spotify': 'https://open.spotify.com/artist/4oUHIQIBe0LHzYfvXNW4QM'},
  'href': 'https://api.spotify.com/v1/artists/4oUHIQIBe0LHzYfvXNW4QM',
  'id': '4oUHIQIBe0LHzYfvXNW4QM',
  'name': 'Morgan Wallen',
  'type': 'artist',
  'uri': 'spotify:artist:4oUHIQIBe0LHzYfvXNW4QM'}]

In [ ]:
data['items'][5]['track']['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/6qqNVTkY8uBg9cP3Jd7DAH'},
  'href': 'https://api.spotify.com/v1/artists/6qqNVTkY8uBg9cP3Jd7DAH',
  'id': '6qqNVTkY8uBg9cP3Jd7DAH',
  'name': 'Billie Eilish',
  'type': 'artist',
  'uri': 'spotify:artist:6qqNVTkY8uBg9cP3Jd7DAH'}]

In [ ]:
artist_list = []
for row in data['items']:
    for key, value in row.items():
        if key == "track":
            for artist in value['artists']:
                artist_dict = {'artist_id':artist['id'], 'artist_name':artist['name'], 'external_url': artist['href']}
                artist_list.append(artist_dict)

In [ ]:
song_list = []
for row in data['items']:
    song_id = row['track']['id']
    song_name = row['track']['name']
    song_duration = row['track']['duration_ms']
    song_url = row['track']['external_urls']['spotify']
    song_popularity = row['track']['popularity']
    song_added = row['added_at']
    album_id = row['track']['album']['id']
    artist_id = row['track']['album']['artists'][0]['id']
    song_element = {'song_id':song_id,'song_name':song_name,'duration_ms':song_duration,'url':song_url,
                    'popularity':song_popularity,'song_added':song_added,'album_id':album_id,
                    'artist_id':artist_id
                   }
    song_list.append(song_element)

In [ ]:
import pandas as pd
album_df = pd.DataFrame.from_dict(album_list)
album_df.head()

,album_id,name,release_date,total_tracks,url
0,6i7mF7whyRJuLJ4ogbH2wh,One Thing At A Time,2023-03-03,36,https://open.spotify.com/album/6i7mF7whyRJuLJ4...
1,5qmZefgh78fN3jsyPPlvuw,a Gift & a Curse,2023-06-16,15,https://open.spotify.com/album/5qmZefgh78fN3js...
2,1NAmidJlEaVgA3MpcPFYGq,Lover,2019-08-23,18,https://open.spotify.com/album/1NAmidJlEaVgA3M...
3,5CM66hwjlbZ06LhONWXOAs,Barbie World (with Aqua) [From Barbie The Album],2023-06-23,1,https://open.spotify.com/album/5CM66hwjlbZ06Lh...
4,18NOKLkZETa4sWwLMIm0UZ,UTOPIA,2023-07-28,19,https://open.spotify.com/album/18NOKLkZETa4sWw...


In [ ]:
album_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41 entries, 0 to 47
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   album_id      41 non-null     object
 1   name          41 non-null     object
 2   release_date  41 non-null     object
 3   total_tracks  41 non-null     int64 
 4   url           41 non-null     object
dtypes: int64(1), object(4)
memory usage: 1.9+ KB


In [ ]:
# Check for duplicate album IDs
duplicate_albums = album_df[album_df.duplicated(subset=['album_id'], keep=False)]
if not duplicate_albums.empty:
    print("Duplicate album IDs found:")
    print(duplicate_albums)
else:
    print("No duplicate album IDs found.")

Duplicate album IDs found:
                  album_id                     name release_date  \
0   6i7mF7whyRJuLJ4ogbH2wh      One Thing At A Time   2023-03-03   
4   18NOKLkZETa4sWwLMIm0UZ                   UTOPIA   2023-07-28   
13  18NOKLkZETa4sWwLMIm0UZ                   UTOPIA   2023-07-28   
15  18NOKLkZETa4sWwLMIm0UZ                   UTOPIA   2023-07-28   
18  18NOKLkZETa4sWwLMIm0UZ                   UTOPIA   2023-07-28   
19  6i7mF7whyRJuLJ4ogbH2wh      One Thing At A Time   2023-03-03   
20  151w1FgRZfnKZA9FEcg9Z3                Midnights   2022-10-21   
23  2fenSS68JI1h4Fo296JfGr                 folklore   2020-07-24   
24  2QJmrSgbdM35R67eoGQo4j                     1989   2014-10-27   
34  2QJmrSgbdM35R67eoGQo4j                     1989   2014-10-27   
36  151w1FgRZfnKZA9FEcg9Z3                Midnights   2022-10-21   
41  19DGbjQW4oJmS1TLmr7OH3  Religiously. The Album.   2023-05-12   
42  6i7mF7whyRJuLJ4ogbH2wh      One Thing At A Time   2023-03-03   
48  19DGbjQW4oJmS1TLm

In [ ]:
album_df = album_df.drop_duplicates(subset=['album_id'])

In [ ]:
artist_df = pd.DataFrame.from_dict(artist_list)

In [ ]:
artist_df = artist_df.drop_duplicates(subset=['artist_id'])

In [ ]:

song_df = pd.DataFrame.from_dict(song_list)

In [ ]:
# Check for duplicate song IDs
duplicate_songs = song_df[song_df.duplicated(subset=['song_id'], keep=False)]
if not duplicate_songs.empty:
    print("Duplicate song IDs found:")
    print(duplicate_songs)
else:
    print("No duplicate song IDs found.")

No duplicate song IDs found.
